# Mini Competition: Richter's Predictor

Import modules and data

In [ ]:
# Modules
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
# Import scripts
import helper_functions # various helper functions
import lgb_optimized # Random forest

In [ ]:
# Load data
X, y, X_test = helper_functions.imports()

Data Analysis

# DATA CLEANING (I only edited this section)

In [ ]:
# list of columns that shall be dropped at the end
drop_cols = []

# helper functions
def ids():
    ids = [id for id in globals().keys() if not id.startswith("_")]
    print(ids[5:])
ids()

print(X.shape, 'damage_grade' in X.columns)
data = pd.merge(left=y, right=X, on='building_id', how='inner')
print(data.shape, 'damage_grade' in data.columns) #ok, one column was added

cols = data.columns
for col in cols:
    globals()[col] = col



## `geo_level2` mean encoding

In [ ]:
geo_levels = []
for col in filter(lambda col: col.startswith("geo"), X):
    print(str(col))
    geo_levels.append(col)

In [ ]:
mean_encodings = data.groupby('geo_level_2_id')['damage_grade'].mean()
pd.DataFrame(mean_encodings).head()

In [ ]:
for df in X, X_test:
    df["geo_level_2_enc"] = df['geo_level_2_id'].map(mean_encodings)
    print(df[["geo_level_2_id", "geo_level_2_enc"]].sort_values("geo_level_2_id")[:100], end="\n")

print(geo_levels)
drop_cols.extend(geo_levels)

## drop categorical variables for now, I will encode them next

In [ ]:
object_dtype_cols = [
    "land_surface_condition", "foundation_type", "roof_type", "ground_floor_type", 
    "other_floor_type", "position", "plan_configuration", "legal_ownership_status"
]

drop_cols.extend(object_dtype_cols)

In [ ]:
# Outliers 

## Drop columns and split data

In [ ]:
print(drop_cols)
for df in X, X_test:
    df.drop(columns=drop_cols, inplace=True)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
# X_train.columns

Visualization

Generation of Random Forest and optimization

In [ ]:
#FOR NOW (to test tree gen) limit dataset to a few parameters only
mask = ['has_superstructure_adobe_mud', 'age','count_floors_pre_eq','area_percentage','height_percentage','has_secondary_use',
        'has_superstructure_cement_mortar_brick', 'has_superstructure_timber', 'has_superstructure_bamboo'] 


# apply mask
X = X[mask]

X_train = X_train[mask]
X_valid = X_valid[mask]

# Cut off 'building_id' from X_test!
X_test_pred = X_test[mask]

In [ ]:
# Initiate model
model = lgb_optimized.LGBM(X_train, X_valid, y_train, y_valid)
# Generate and optimize Random Forest, feed with test data to create prediction

y_pred = model.optimization(X, y, X_test_pred)

Evaluation

Export

In [ ]:
# Export prediction to creat csv by DrivenData.org datastandards
helper_functions.write_output(X_test, y_pred)